In [57]:
import PyPDF2
import numpy as np
import pandas as pd

In [135]:
def read_statement(page):
    text = page.extract_text()
    if not 'Bank Statement Nr.' in text:
        return pd.DataFrame(columns = ["Merchant", "Category", "Date", "Amount"])
    # get date of statement
    lines = text.split("\n")
    date = lines[4].split()[1]

    # remove first 5 lines of text and reassemble text
    lines = lines[6:]
    text = '\n'.join(lines)

    # find entry of each income or expenditure
    entries = [i for i in text.split("€") if i]
    entries = [list(filter(None,i.split("\n"))) for i in entries if i]
    entries = entries[:-1]

    # remove movement between spaces
    remove_list = []
    for entry in entries:
        if len(entry) < 3:
            remove_list.append(entry)
    for entry in remove_list:
        entries.remove(entry)

    #  remove irrelevant information and parse transaction date and amount
    for entry in entries:
        if entry[1].startswith('Mastercard • '):
            entry[1] = entry[1].replace('Mastercard • ','')
        for item in entry[2:]:
            if not item.startswith('Value Date '):
                entry.remove(item)
        entry[2] = entry[2][21:]
        splitted = entry[2].split()
        entry.pop(2)
        entry.append(splitted[0])
        entry.append(splitted[1])
        entry[3] = entry[3].replace('.','')
        entry[3] = entry[3].replace(',','.')
        entry[3] = entry[3].replace('+','')
        if len(entry) != 4:
            print(entry)

    # now only 4 columns are left, merchant/payee, category, date and amount
    # add to pandas
    entries = np.array(entries)
    df = pd.DataFrame(entries, columns = ["Merchant", "Category", "Date", "Amount"])
    df["Amount"] = df["Amount"].astype('float')

    return(df)

In [190]:
reader = PyPDF2.PdfReader("statement-2023-09.pdf")
df = pd.DataFrame(columns = ["Merchant", "Category", "Date", "Amount"])
for page in reader.pages:
    df = pd.concat([df, read_statement(page)])

# fix some category assignment issues
df.loc[df["Merchant"].str.contains('STEAMGAMES'), "Category"] = 'Games'
df.loc[df["Merchant"].str.contains('APPLE'), "Category"] = 'Games'
df.loc[df["Merchant"].str.contains('AMAZON'), "Category"] = 'Amazon'
df.loc[df["Merchant"].str.contains('UBER \*EATS'), "Category"] = 'Bars & Restaurants'

# summarize income
income = df[df["Amount"] > 0]
summary_income = income.groupby("Merchant", as_index = False).agg({"Amount": sum})
total_income = income["Amount"].sum()

# summarize expenditure
expenditure = df[df["Amount"] < 0]
total_expenditure = -expenditure["Amount"].sum()
summary_expenditure = expenditure.groupby("Category", as_index = False).sum()[["Category", "Amount"]]
summary_expenditure["Amount"] = -summary_expenditure["Amount"]

/var/folders/tm/6h1pp8k1183g0792lm0b20s00000gp/T/ipykernel_57717/3374457670.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, read_statement(page)])
/var/folders/tm/6h1pp8k1183g0792lm0b20s00000gp/T/ipykernel_57717/3374457670.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, read_statement(page)])
/var/folders/tm/6h1pp8k1183g0792lm0b20s00000gp/T/ipykernel_57717/3374457670.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

In [193]:
print("Total income:", total_income)
print("Breakdown:")
print(summary_income)
print()
print("Total expenditure:", total_expenditure)
print("Breakdown:")
print(summary_expenditure)
print()

category = "Direct Debits"
print("Expenditures from category", category)
print(expenditure[expenditure["Category"] == category])



Total income: 3363.51
Breakdown:
                        Merchant   Amount
0  AMAZON PAYMENTS EUROPE S.C.A.    17.99
1     Boehringer Ingelheim Fonds  2201.85
2           Dana - Maria Grozavu    97.68
3                           EMBL   345.79
4                     HENNER-GMC   491.40
5                    Yidan Jiang   208.80

Total expenditure: 2005.14
Breakdown:
               Category  Amount
0                   ATM  200.00
1                Amazon   58.13
2    Bars & Restaurants  209.61
3     Business Expenses   15.90
4         Direct Debits  331.62
5                 Games  123.45
6             Groceries  250.35
7  Multimedia & Telecom    7.99
8    Outgoing Transfers  796.30
9              Shopping   11.79

Expenditures from category Direct Debits
                                            Merchant       Category  \
0                          Rhein-Neckar-Verkehr GmbH  Direct Debits   
3                         EnBW Energie Bad-Wuertt AG  Direct Debits   
0                          